<a href="https://colab.research.google.com/github/maferrara1961/colab/blob/main/fourier3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🌌 Análisis Espectral: Full Data Science Edition
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.fftpack import fft, fftfreq
from IPython.display import HTML, display, Audio
import os
import urllib.request
import librosa
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# ==========================================
# 1. CONFIGURACIÓN VISUAL
# ==========================================
display(HTML("""
<div style="background:linear-gradient(135deg,#0d0d1a,#1a0033,#330033);
            color:#fff; padding:20px; border-radius:20px; font-family:'Courier New', monospace;
            box-shadow:0 0 40px rgba(255,0,255,0.8); text-align:center;">
  <h1 style="color:#00ffff; text-shadow:0 0 20px #00ffff; margin:0;">
    🌌 Universo Sonoro: SETI Edition - Señal WOV! 🌌
  </h1>
  <p style="color:#ffccff; font-size:14px;">Analizando señal interceptada...</p>
</div>
"""))

plt.style.use('dark_background')
plt.rcParams.update({
    'axes.facecolor': '#1a0033', 'figure.facecolor': '#0d0d1a',
    'grid.color': '#440044', 'text.color': '#00ffff',
    'axes.labelcolor': '#ffccff', 'xtick.color': '#ffccff', 'ytick.color': '#ffccff',
    'font.family': 'sans-serif'
})

# ==========================================
# 2. CARGA DE DATOS
# ==========================================
url_origen = "https://www.seti.net/indepth/wow/2009-03-12T11-25-06.WAV"
nombre_archivo = "senal_final.wav"

if not os.path.exists(nombre_archivo):
    try:
        urllib.request.urlretrieve(url_origen, nombre_archivo)
    except:
        import scipy.io.wavfile
        t_dummy = np.linspace(0, 1, 44100)
        scipy.io.wavfile.write(nombre_archivo, 44100, (np.sin(2*np.pi*440*t_dummy)*0.5).astype(np.float32))

# Carga
senal_audio, fs = librosa.load(nombre_archivo, sr=None, mono=True)
tiempo = np.arange(len(senal_audio)) / fs

print(f"\n🎧 Reproductor ({len(senal_audio)} muestras):")
display(Audio(senal_audio, rate=fs))

# ==========================================
# 3. ANÁLISIS ESTADÍSTICO (DESCRIBE)
# ==========================================
print("\n📊 REPORTE ESTADÍSTICO DE LA ONDA (Describe)")
print("=" * 60)

# Crear Serie de Pandas
serie_datos = pd.Series(senal_audio)
stats = serie_datos.describe()

# Cálculos extra de ingeniería de audio
duracion = len(senal_audio) / fs
rms = np.sqrt(np.mean(senal_audio**2)) # Root Mean Square (Energía real)
dc_offset = np.mean(senal_audio)       # Desviación del centro
rms = np.sqrt(np.mean(senal_audio**2))
peak_pos = np.max(senal_audio)
peak_neg = np.min(senal_audio)

print(f"⏱️  Duración        : {len(senal_audio)/fs:.4f} segundos")
print(f"⚡  Energía RMS     : {rms:.6f}")
print(f"📈  Rango Dinámico  : {peak_neg:.4f} a {peak_pos:.4f}")
print("-" * 60)
print(stats.to_string(float_format="{:.6f}".format))
print("=" * 60)

# Interpretación automática
if abs(dc_offset) > 0.01:
    print("⚠️ ALERTA: Se detectó un desplazamiento DC significativo (Ruido eléctrico).")
else:
    print("✅ CALIDAD: La señal está bien centrada.")

# ==========================================
# 4. GRÁFICO 1: OSCILOSCOPIO (TIEMPO)
# ==========================================
fig_time = go.Figure()

fig_time.add_trace(go.Scatter(
    x=tiempo, y=senal_audio,
    mode='lines', name='Amplitud',
    line=dict(color='#00ffff', width=1), opacity=0.9
))

fig_time.update_layout(
    title="<b>🌊 OSCILOSCOPIO TEMPORAL (Rango Ajustable)</b>",
    xaxis_title="Tiempo (segundos)",
    yaxis_title="Amplitud",
    template="plotly_dark",
    paper_bgcolor='#0d0d1a', plot_bgcolor='#1a0033',
    height=500,
    font=dict(family="Courier New, monospace", color="#ffccff"),

    # Range Slider para elegir el rango de tiempo
    xaxis=dict(
        rangeslider=dict(visible=True, thickness=0.1),
        type="linear"
    )
)
fig_time.show()

# ==========================================
# 4. VISUALIZACIÓN TEMPORAL
# ==========================================
tiempo = np.arange(len(senal_audio)) / fs
print("\n🌊 Gráfica 1: Histograma y Onda")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Plot Onda
ax1.plot(tiempo, senal_audio, color='#00ffff', linewidth=0.2, alpha=0.8)
ax1.set_title("OSCILOSCOPIO", color="#ff00ff", fontweight="bold")
ax1.set_xlabel("Tiempo (s)")
ax1.set_ylabel("Amplitud")
ax1.grid(True, linestyle='--', alpha=0.3)

# Plot Histograma (Distribución de amplitud)
ax2.hist(senal_audio, bins=50, color='#ff00ff', alpha=0.7, rwidth=0.9)
ax2.set_title("HISTOGRAMA DE AMPLITUD", color="#ff00ff", fontweight="bold")
ax2.set_xlabel("Amplitud")
ax2.set_ylabel("Frecuencia (Conteo)")
ax2.grid(True, linestyle='--', alpha=0.3)
ax2.set_yscale('log') # Escala logarítmica para ver mejor los detalles

plt.show()

# ==========================================
# 5. CÁLCULO FFT & ESCALADO MAGNITUD
# ==========================================
N = len(senal_audio)
transformada = fft(senal_audio)
modulo = np.abs(transformada)[:N//2] * (2/N)
frecuencias = fftfreq(N, 1/fs)[:N//2]

# Buscar Máximos
idx_max = np.argmax(modulo)
freq_principal = frecuencias[idx_max]
magnitud_maxima = modulo[idx_max]

# --- LÓGICA DE ESCALADO + 10% ---
limite_superior_y = magnitud_maxima * 1.0001
# --------------------------------

print("\n")
print("-" * 50)
print(f"🚀 FRECUENCIA DOMINANTE: {freq_principal:.2f} Hz")
print(f"📏 MAGNITUD MÁXIMA    : {magnitud_maxima:.6f}")
#print(f"🔭 ESCALA GRÁFICO (+10%): 0 a {limite_superior_y:.6f}")
print("-" * 50)

# ==========================================
# 6. GRÁFICO 2: FOURIER (AGRANDADO Y ESCALADO)
# ==========================================
fig_freq = go.Figure()

fig_freq.add_trace(go.Scatter(
    x=frecuencias, y=modulo,
    mode='lines', name='Espectro',
    fill='tozeroy', line=dict(color='#ff00ff', width=1.5)
))

# Anotación del Pico
fig_freq.add_annotation(
    x=freq_principal, y=magnitud_maxima,
    text=f"⭐ PICO: {freq_principal:.1f} Hz",
    showarrow=True, arrowhead=2, arrowcolor="#ffff00",
    ax=0, ay=-40,
    font=dict(color="#ffff00", size=14, family="Courier New"),
    bgcolor="rgba(0,0,0,0.5)", bordercolor="#ffff00"
)

fig_freq.update_layout(
    title=f"<b>🌌 TRANSFORMADA DE FOURIER (Escala Automática al Pico)</b>",
    xaxis_title="Frecuencia (Hz)",
    yaxis_title="Magnitud",
    template="plotly_dark",
    paper_bgcolor='#0d0d1a', plot_bgcolor='#1a0033',
    height=700, # Gráfico Agrandado
    font=dict(family="Courier New, monospace", color="#ffccff"),

    # Zoom inicial en X
    xaxis=dict(range=[0, 5000]),

    # AQUI APLICAMOS EL ESCALADO + 10%
    yaxis=dict(range=[0, limite_superior_y])
)

fig_freq.show()